In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
learning_rate = 3e-3
max_iters = 100000
eval_iters = 250

cuda


In [2]:
from google.colab import files
file = files.upload()
with open('On War Intro 1_10.txt', 'r', encoding = 'utf-8') as f:
  text = f.read()
  print(text[:200])

Saving On War Intro 1_10.txt to On War Intro 1_10.txt


THE INTRODUCTION OF THE AUTHOR

That the conception of the scientific does not consist alone, or
chiefly, in system, and its finished theoretical constructions,
requires nowadays no exposition. Syst


In [3]:
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
print('len chars:', len(chars))

['\n', ' ', '&', '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ü', 'œ', '—', '’', '“', '”']
len chars: 80


In [4]:
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i :ch for i, ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l : ''.join([int_to_string[i] for i in l])

In [5]:
data = torch.tensor(encode(text), dtype = torch.long)
print(len(data))


30368


In [6]:
# test train splits
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
x = train_data[:block_size]
y = train_data[1: block_size + 1]
for t in range(block_size):
  print('t:', t)
  context = x[:t+1]
  target = y[t]
  print('when input is', context, 'target is', target)

In [ ]:

# # test train splits
# n = int(0.8*len(data))
# train_data = data[:n]
# val_data = data[n:]

# how we create batches of data
def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,)) # randomiszed indices for generation
  x = torch.stack([data[i: i+ block_size] for i in ix]) # create a tensor by slecting the randomly generated intergers created in ix
  y = torch.stack([data[i + 1: i + block_size + 1]for i in ix]) # creating batches of y
  x, y = x.to(device), y.to(device)
  return x, y


x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

In [9]:
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

In [12]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  #(B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1)  # (B, 1)

            # Ensure that the generated indices are within the valid range
            index_next = torch.clamp(index_next, 0, vocab_size - 1)

            # append sample index to the running index
            index = torch.cat((index, index_next), dim=1)  # (B, T + 1)
        return index

vocab_size = len(chars)  # Replace with your vocabulary size
model = BigramLanguageModel(vocab_size)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(model.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



Vsxldy5NTr&fJiz
N:i
Old&G5ki3_Lu4:obRyZPw&sü?B6—LKXœv:x2MV.7k’nOA:v“o30IwjobRm-(HAn5B5pEWVüFXIw3pNkzBxC.üuZ6—*ü,nO)jh*vH&c.kr;“4EKjVhJqFn’(:TldXz1_gL5UgMDope9T—8
NE*eœWR—’
N;AV2WRdpG6Oy0PtKbE )dd(AHM*ü7Fi1luJAAmmT,h)u9.0b’Hj“h—5;fK:œoMuk’05AMFUJ’o?-T1NxdMFrwjM&wvük’*TsGHDL5L4bR)œ8Xj“96roL-XükUdüj4iVBRDld”lkS7ZL-h.37i3_Pe1qex5gMuU_q’ZJsLMPN’H0xT,I7Müq“—oIo5Xvxyœ4B;—Ex2NyYfyK1-peJKyVdsU)j*X17gv._bXRBC&8Tt’eRnO—wTh8hr,TDqA VM5g2yNNYpao5??L5R)(6Wk’b8Za?RopGU-J—fb7VFah;Rbdm—m&U8t0P&40“üsü5’_mRC.ykm)S


In [ ]:
# create pythorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):
  if iter %eval_iters == 0:
    losses = estimate_loss()
    print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")



  # sample a batch of data
  xb, yb = get_batch('train')


  #evaluate the loss
  logits, loss = model.forward(xb, yb)
  optimizer.zero_grad(set_to_none = True)
  loss.backward()
  optimizer.step()
print(loss.item())


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Generate some random training data
# For simplicity, we'll use a small vocabulary and generate sequences of integers.
# In a real application, you'd replace this with your actual training data.
vocab_size = 10
sequences = [
    [1, 2, 3, 4, 5],
    [2, 3, 4, 5, 6],
    [3, 4, 5, 6, 7],
    [4, 5, 6, 7, 8],
]

# Define a simple bigram language model
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.fc = nn.Linear(embedding_dim, vocab_size)

    def forward(self, x):
        embedded = self.embedding(x)
        out = self.fc(embedded)
        return out

# Define hyperparameters
embedding_dim = 16
learning_rate = 0.01
epochs = 100

# Create the model
model = BigramLanguageModel(vocab_size, embedding_dim)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    total_loss = 0.0
    for sequence in sequences:
        inputs = torch.LongTensor(sequence[:-1])  # Input sequence (exclude the last word)
        targets = torch.LongTensor(sequence[1:])   # Target sequence (exclude the first word)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Compute loss
        loss = criterion(outputs, targets)

        # Backpropagation and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Print the average loss for this epoch
    avg_loss = total_loss / len(sequences)
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}")

# Example usage: Generate a sequence given a starting word
starting_word = torch.LongTensor([1])  # Replace with your desired starting word's index
generated_sequence = [starting_word.item()]

for _ in range(4):  # Generate a 5-word sequence
    outputs = model(starting_word)
    _, next_word = torch.max(outputs, dim=1)
    generated_sequence.append(next_word.item())
    starting_word = next_word

print("Generated Sequence:", generated_sequence)
